In [ ]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [ ]:
!pip install spacy
!pip install benepar
!pip install cython numpy
!pip install benepar[cpu]
!python -m spacy download en

import nltk
import spacy
import benepar
from benepar.spacy_plugin import BeneparComponent
benepar.download('benepar_en2')
from spacy import displacy
from nltk import Tree
nltk.download('punkt')


     |████████████████████████████████| 81kB 3.6MB/s 
  Created wheel for benepar: filename=benepar-0.1.2-cp36-cp36m-linux_x86_64.whl size=104446 sha256=08b802b41f4688f065cffafa947d423ca8129c9e8f1c167bf1569db9bd0d302e
  Stored in directory: /root/.cache/pip/wheels/c6/f5/06/d88543b19a9b326007d7538298a139e994b1d2eecb003bf5af
Successfully built benepar
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7542 sha256=425b926853b83ea9dca5e98ffaa19ae973b5d4c4e3bd81fc337aa00164e0650b
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model 

True

In [ ]:
#Constituency parsing using Berkeley Neural Parser
nlp = spacy.load('en')
nlp.add_pipe(BeneparComponent("benepar_en2"))
doc = nlp(u"Ram stayed at home in the evening.")

In [ ]:
#CP tree
parser = benepar.Parser("benepar_en2")
tree = parser.parse("Ram and Jim stayed at home in the evening")
print(tree)

(S
  (NP (NNP Ram) (CC and) (NNP Jim))
  (VP
    (VBD stayed)
    (PP (IN at) (NP (NN home)))
    (PP (IN in) (NP (DT the) (NN evening)))))


Dependency Parsing begins

In [ ]:
nlp = spacy.load('en')
nlp.add_pipe(BeneparComponent("benepar_en2"))
doc = nlp(u"Ram and Jim stayed at home in the evening")

In [ ]:
#Dependency parsing 

for token in doc:
 print(token.text,'=>',token.dep_,'=>',token.head.text)

displacy.render(doc, style='dep',jupyter=True)

Ram => nsubj => stayed
and => cc => Ram
Jim => conj => Ram
stayed => ROOT => stayed
at => prep => stayed
home => pobj => at
in => prep => stayed
the => det => evening
evening => pobj => in


In [ ]:
for token in doc:
  if token.dep_ == 'ROOT':
    root = token
  if token.dep_ == 'nsubj':
    nSub = token

root_connects=[]
for token in doc:
  if token.head==root:
      root_connects.append(token)

print(root, root.tag_)
print(nSub, nSub.tag_)

root_connects.remove(root)
root_connects.remove(nSub)
print(root_connects)

level2 = []
for token in doc:
  for word in root_connects:
    if token.head == word:
      level2.append(token)

print(level2)

level3 = []
for token in doc:
  for word in level2:
    if token.head == word:
      level3.append(token)

print(level3)

stayed VBD
Ram NNP
[at, in]
[home, evening]
[the]


In [ ]:
print('S')

print('\t NP')
print('\t\t',nSub.tag_, nSub)
if len([child for child in nSub.children]) > 0:
  for child in nSub.children:
    print('\t\t\t', child.tag_, child)


print('\t VP')

print('\t\t', root.tag_, root)
for word in root_connects:
  if len([child for child in word.children]) > 0:
    if word.tag_ == 'NN':
      print('\t\t NP')
    elif word.head.tag_ == 'PP' or 'IN':
      print('\t\t pp')
    print('\t\t', word.tag_, word,)
    for child in word.children:
      if len([child1 for child1 in child.children]) > 0:
        if child.tag_ == 'NN':
          print('\t\t\t NP')
        elif word.head.tag_ == 'PP' or 'IN':
          print('\t\t\t pp')
        print('\t\t\t', child.tag_, child)
        for child1 in child.children:
           print('\t\t\t\t', child1.tag_, child1.text )
        break
      print('\t\t\t', child.tag_, child.text )

#print('\t\t', word.tag_, word, [child for child in word.children])


S
	 NP
		 NNP Ram
			 CC and
			 NNP Jim
	 VP
		 VBD stayed
		 pp
		 IN at
			 NN home
		 pp
		 IN in
			 NP
			 NN evening
				 DT the
